In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import joblib

In [2]:
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension)
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,input_image.shape[0])
    pca_return_array = np.array(pca_return_array)
    #print (pca.explained_variance_ratio_)
    return pca_return_array

In [3]:
def trans(A):
    A_c = A.transpose(2,1,0)
    A_c = A_c.transpose(0,2,1)
    return A_c

In [4]:
# 可以提取出高光谱数据的一个channel的数据 145*145
# 注意这里输入转换坐标后的矩阵
def split_one_channel(A,channel):
    return A[channel-1] 
# 参数说明
# channel 通道
# A 高光谱立方块

# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


# 将镜像处理后的矩阵进行向量化操作
def split_array(A,fil):
    s = []
    row = len(A)-fil+1
    col = len(A[0])-fil+1
    for i in range(row): # row
        for j in range(col): # col
            temp = [0]* fil
            for z in range(fil):
                temp[z] = A[i+z,j:j+fil]
            com = np.concatenate(temp)
            s.append(com)
    return s 
# 参数说明
# A 镜像处理后的矩阵
# fil  filter size


# 计算欧几里得距离
def Euclidean(v1,v2):
    return np.linalg.norm(v1-v2)


# 计算单个patch的空间信息
def patch_spitial(v1, K_centroid):
    distance_array = np.array([Euclidean(v1, centroid) for centroid in K_centroid]) #距离matrix
    sums = np.sum(distance_array)
    average = np.average(distance_array)
    spitial_one_patch = np.maximum(0,average - distance_array) #依据公式求空间信息
    return spitial_one_patch

# 对一个channel的求解进行向量化
def channel_spitial(channel_matrix, K_centroid):
    channel_spitial_list = list()
    for i in range(len(channel_matrix)):
        channel_spitial_list.append(patch_spitial(channel_matrix[i], K_centroid))
    return np.array(channel_spitial_list)

In [5]:
# 构造n个通道的向量 输入转换坐标后的矩阵
def create_whole_vector(dim,fil,input_image):
    # 生成器
    pad =int((fil-1)/2)
    s = (split_one_channel(input_image,i) for i in range(1,dim+1))
    whole_origin_matrix_array = np.array(next(s))
    while True:
        try:
            whole_origin_matrix_array = np.vstack((whole_origin_matrix_array,next(s)))
        except StopIteration as e:
            whole_origin_matrix_array = whole_origin_matrix_array.reshape(dim,input_image.shape[1],
                                                                          input_image.shape[2])                                                                     
            break
    p = (mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim))
    whole_mirror_matrix_array = np.array(next(p))
    while True:
        try:
            whole_mirror_matrix_array = np.vstack((whole_mirror_matrix_array, next(p)))
        except StopIteration as e:
            whole_mirror_matrix_array = whole_mirror_matrix_array.reshape(dim,input_image.shape[1]+pad*2,
                                                                          input_image.shape[2]+pad*2)
            break
    t = (split_array(whole_mirror_matrix_array[i],fil) for i in range(dim))
    whole_split_matrix_array = np.array(next(t))
    while True:
        try:
            whole_split_matrix_array = np.vstack((whole_split_matrix_array, next(t)))
        except StopIteration as e:
            whole_split_matrix_array = whole_split_matrix_array.reshape(dim,input_image.shape[1]*input_image.shape[2],-1)
            break
    return  whole_split_matrix_array



## 通过k-means求解 whole_K_centroid
K = 5
def create_centroid(K,dim,whole_split_matrix_array):
    whole_kmeans = [KMeans(n_clusters=K, random_state=0).fit(whole_split_matrix_array[i]) for i in range(dim)] 
    t = (whole_kmeans[i].cluster_centers_ for i in range(dim))
    whole_K_centroid = np.array(next(t))
    while True:
        try:
            whole_K_centroid = np.vstack((whole_K_centroid,next(t)))
        except StopIteration as e:
            whole_K_centroid = whole_K_centroid.reshape(dim,K,-1)
            break
    
    return whole_K_centroid

## 求整体空间信息
def create_spe_spi(whole_K_centroid,whole_split_matrix_array,dim):
    g = (channel_spitial(whole_split_matrix_array[i], whole_K_centroid[i])for i in range(whole_K_centroid.shape[0]))
    whole_spitial  = np.array(next(g))
    while True:
        try:
            whole_spitial = np.vstack((whole_spitial, next(g)))
        except StopIteration as e:
            whole_spitial = whole_spitial.reshape(dim,whole_split_matrix_array.shape[1],-1)
            break
    return whole_spitial

###### 可选操作，将spectral-spitial信息以numpy形式保存
def save_spe_spi(whole_spitial,path='I:\data\whole_spitial.npy'):
    np.save(path,whole_spitial)
###### 可选操作，载入数据
def load_spe_spi(name):
    s = np.load(name)
    return s

## 合成sample_with_lables
def sample_with_lables(s,dim,output_image):
    sample = np.hstack([s[i]for i in range(dim)])
    label = output_image.reshape(len(sample),1)
    sample_lb = np.hstack([sample,label])
    return sample_lb


## 提取出需要分类的数据
def need_claasify(sample_lb):
    sample_lab_fix_list = list()
    for i in range(len(sample_lb)):
        if sample_lb[i,-1] != 0:
            sample_lab_fix_list.append(sample_lb[i,:])
    sample_lab_fix = np.array(sample_lab_fix_list)
    return sample_lab_fix



## 将数据归一化并且存储：
def scaler_save(sample_lab_fix, path = 'H:\data\spec_spi.csv'):
    ## 归一化
    data_D = preprocessing.StandardScaler().fit_transform(sample_lab_fix[:,:-1])
    data_L = sample_lab_fix[:,-1]

    ## 存储
    new = np.column_stack((data_D, data_L))
    new_ = pd.DataFrame(new)
    new_.to_csv(path,header=False,index=False)
    

## 分类数据提取
def classify_pre(path='H:\data\spec_spi.csv'):
    data = pd.read_csv(path,header=None)
    data = data.as_matrix()
    data_D = data[:,:-1]
    data_L = data[:,-1]
    data_train, data_test, label_train, label_test = train_test_split(data_D,data_L,test_size=0.9)
    return data_train, data_test, label_train, label_test

### 分类进行

In [6]:
### 设置初始值
DIM = 30
FIL = 50
K = 5

## 载入数值
input_image = loadmat('I:\data\Salinas_corrected.mat')['salinas_corrected']
output_image = loadmat('I:\data\Salinas_gt.mat')['salinas_gt']

In [7]:
# pca后 将数据的光谱放到第一维
input_image_pca = pca_hpi(input_image, DIM)
input_image_pca_trans = trans(input_image_pca)
input_image_pca_trans.shape

(30, 512, 217)

In [8]:
# 将数据全部向量化
whole_split_matrix_array =create_whole_vector(DIM,FIL,input_image_pca_trans)
whole_split_matrix_array.shape

MemoryError: 